In [1]:
!pip install XGBoost
!pip install catboost
!pip install optuna
!pip install LightGBM
!pip install optuna-integration[lightgbm]
import pandas as pd
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 4.3 MB/s eta 0:00:00


In [7]:
from sklearn.model_selection import train_test_split
df_m_train = pd.read_csv("train_Mortgage.csv")
df_p_train = pd.read_csv("train_Pension.csv")
df_s_train = pd.read_csv("train_Savings.csv")
target_name_1 = "Mortgage"
target_name_2 = "Pension"
target_name_3 = "Savings"
y_train_m = df_m_train[target_name_1]
y_train_p = df_p_train[target_name_2]
y_train_s = df_s_train[target_name_3]
X_train_m = df_m_train.drop(columns=[target_name_1])
X_train_p = df_p_train.drop(columns=[target_name_2])
X_train_s = df_s_train.drop(columns=[target_name_3])

In [8]:
df_m_valid = pd.read_csv("val_Mortgage.csv")
df_p_valid = pd.read_csv("val_Pension.csv")
df_s_valid = pd.read_csv("val_Savings.csv")
y_test_m = df_m_valid[target_name_1]
y_test_p = df_p_valid[target_name_2]
y_test_s = df_s_valid[target_name_3]
X_test_m = df_m_valid.drop(columns=[target_name_1])
X_test_p = df_p_valid.drop(columns=[target_name_2])
X_test_s = df_s_valid.drop(columns=[target_name_3])

In [4]:
X_train_m, X_valid_m, y_train_m, y_valid_m = train_test_split(X_train_m, y_train_m, test_size = 0.15, stratify=y_train_m)
X_train_p, X_valid_p, y_train_p, y_valid_p = train_test_split(X_train_p, y_train_p, test_size = 0.15, stratify=y_train_p)
X_train_s, X_valid_s, y_train_s, y_valid_s = train_test_split(X_train_s, y_train_s, test_size = 0.15, stratify=y_train_s)

CatBoost + ExtraTrees + LightGBM + XGBoost - > LogisticRegression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Базовые модели
models = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

def evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model):
    # Создаем стекинг
    stacking_model = StackingClassifier(
        estimators=list(models.items()),
        final_estimator=meta_model,
        cv=5,
        stack_method="predict_proba"
    )

    # Обучение и предсказание
    stacking_model.fit(X_train, y_train)
    y_pred = stacking_model.predict(X_test)
    y_proba = stacking_model.predict_proba(X_test)[:, 1]  # для ROC-AUC

    # Оценка
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    return accuracy, roc_auc

# Мета-модель
meta_model = LogisticRegression()

# Проверяем на каждом таргете
targets = {
    "Mortgage": (X_train_m, X_test_m, y_train_m, y_test_m),
    "Pension": (X_train_p, X_test_p, y_train_p, y_test_p),
    "Savings": (X_train_s, X_test_s, y_train_s, y_test_s)
}

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7598, ROC-AUC = 0.9395
Pension: Accuracy = 0.5540, ROC-AUC = 0.7569
Savings: Accuracy = 0.5521, ROC-AUC = 0.6725


In [6]:
import pandas as pd
from itertools import combinations
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC

# Загрузка данных
def load_data(target_name, train_path, val_path):
    df_train = pd.read_csv(train_path)
    df_val = pd.read_csv(val_path)
    y_train = df_train[target_name]
    y_test = df_val[target_name]
    X_train = df_train.drop(columns=[target_name])
    X_test = df_val.drop(columns=[target_name])
    return X_train, X_test, y_train, y_test

# Данные для каждого таргета
targets = {
    "Mortgage": ("train_Mortgage.csv", "val_Mortgage.csv"),
    "Pension": ("train_Pension.csv", "val_Pension.csv"),
    "Savings": ("train_Savings.csv", "val_Savings.csv")
}

# Все модели для комбинирования
all_models = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1, random_state=42),
    "XGBoost": XGBClassifier(verbosity=0, random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

# Мета-модель
meta_model = LogisticRegression()

# Перебираем все комбинации моделей от 2 до 6
min_comb_size = 2
max_comb_size = len(all_models)

results = {}

for target_name, (train_path, val_path) in targets.items():
    X_train, X_test, y_train, y_test = load_data(target_name, train_path, val_path)
    target_results = []

    for r in range(min_comb_size, max_comb_size + 1):
        for model_names in combinations(all_models.keys(), r):
            models = {name: all_models[name] for name in model_names}

            stacking = StackingClassifier(
                estimators=list(models.items()),
                final_estimator=meta_model,
                cv=5,
                stack_method='predict_proba'
            )

            stacking.fit(X_train, y_train)
            y_proba = stacking.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_proba)

            target_results.append({
                "Models": "+".join(model_names),
                "ROC-AUC": auc,
                "Num_Models": r
            })


    # Сохраняем результаты для таргета
    results[target_name] = pd.DataFrame(target_results).sort_values("ROC-AUC", ascending=False)

# Выводим лучшие комбинации для каждого таргета
for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.9407109812515217), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.9406839271704137), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.9410717023329636), 'Num_Models': 2}
{'Models': 'CatBoost+RandomForest', 'ROC-AUC': np.float64(0.9412340268196124), 'Num_Models': 2}
{'Models': 'CatBoost+SVM', 'ROC-AUC': np.float64(0.941152864576288), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.937762086410735), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.9378973568162758), 'Num_Models': 2}
{'Models': 'LightGBM+RandomForest', 'ROC-AUC': np.float64(0.9385286187087989), 'Num_Models': 2}
{'Models': 'LightGBM+SVM', 'ROC-AUC': np.float64(0.9379334289244198), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.9340015691367043), 'Num_Models': 2}
{'Models': 'XGBoost+RandomForest', 'ROC-AUC': np.float64(0.9343262181100019), 'Nu

In [23]:
import pandas as pd
from itertools import combinations
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier

# Все модели для комбинирования

cb_mortgage = CatBoostClassifier(iterations=839, learning_rate=0.05774795442279825, depth=6,
                                 l2_leaf_reg=0.2880865175456189, random_strength=0.16254277485904814,
                                 bagging_temperature=0.7992786757360438, border_count=32, boosting_type='Ordered', verbose=0)
xgb_mortgage = XGBClassifier(n_estimators=234, max_depth=14, learning_rate=0.13020051661181956,
                                 subsample=0.5000183482043092, colsample_bytree=0.9712108470499956,
                                 min_child_weight=10, gamma=4.927111346239827, reg_alpha=2.3159564553387995,
                                 reg_lambda=1.7788240357504277, scale_pos_weight=1.155068929743032)

lgb_mortgage = LGBMClassifier(n_estimators=891, learning_rate=0.2775178145806826, num_leaves=154, max_depth=4,
                                  min_child_samples=5, subsample=0.875834146327149, colsample_bytree=0.9367342318343456,
                                  reg_alpha=4.0997572412343555, reg_lambda=1.3718029339702882, verbose=-1)

et_mortgage = ExtraTreesClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=2, n_estimators=200)
lr_mortgage = LogisticRegression(C=0.01, max_iter=1000, solver='lbfgs')




cb_pension = CatBoostClassifier(iterations=200, learning_rate=0.010875325239675968, l2_leaf_reg=0.010817134192675052,
                                random_strength=4.740509717654505, bagging_temperature=0.9584104224319896,
                                border_count=75, colsample_bylevel=0.25113614025415143, depth=1, boosting_type='Plain', verbose=0)
xgb_pension = XGBClassifier(n_estimators=238, max_depth=3, learning_rate=0.058427369161116946,
                                subsample=0.9293867219852593, colsample_bytree=0.5238187849054002,
                                min_child_weight=6, gamma=4.7308173402905505, reg_alpha=8.066467399907712,
                                reg_lambda=4.390801165747763, scale_pos_weight=1.0066125076750385)
lgb_pension = LGBMClassifier(n_estimators=1191, learning_rate=0.11153678049934727, num_leaves=107, max_depth=3,
                                 min_child_samples=41, subsample=0.5173705475484477, colsample_bytree=0.7026966245620551,
                                 reg_alpha=0.45638892145814025, reg_lambda=5.315359029396548, verbose=-1)
et_pension = ExtraTreesClassifier(max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=100)
lr_pension = LogisticRegression(C=0.001, max_iter=1000, solver='lbfgs')

cb_savings = CatBoostClassifier(iterations=205, learning_rate=0.041965496183629054, l2_leaf_reg=0.0033091053333706402,
                                random_strength=0.4267643014670958, bagging_temperature=0.6548287821971729, border_count=254,
                                colsample_bylevel=0.6355211623405239, depth=5, boosting_type='Plain', verbose=0)
xgb_savings = XGBClassifier(n_estimators=1446, max_depth=3, learning_rate=0.17633766607373635,
                                subsample=0.8802608079409611, colsample_bytree=0.6399123535237496,
                                min_child_weight=10, gamma=4.7135127571247475, reg_alpha=7.307229547015628,
                                reg_lambda=0.5506026977366328, scale_pos_weight=4.0245802517341716)
lgb_savings = LGBMClassifier(n_estimators=903, learning_rate=0.2620735099496511, num_leaves=198, max_depth=5,
                                 min_child_samples=98, subsample=0.5818744149073714, colsample_bytree=0.9979504029037664,
                                 reg_alpha=3.2759181752443514, reg_lambda=9.427794042824686, verbose=-1)

et_savings = ExtraTreesClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=10, n_estimators=200)
lr_savings = LogisticRegression(C=0.1, max_iter=1000, solver='lbfgs')

all_models_m = {
    "CatBoost": cb_mortgage,
    "LightGBM": lgb_mortgage,
    "XGBoost": xgb_mortgage,
    "ExtraTrees": et_mortgage,
    "LogisticRegression" : lr_mortgage
}

all_models_p = {
    "CatBoost": cb_pension,
    "LightGBM": lgb_pension,
    "XGBoost": xgb_pension,
    "ExtraTrees": et_pension,
    "LogisticRegression" : lr_pension
}

all_models_s = {
    "CatBoost": cb_savings,
    "LightGBM": lgb_savings,
    "XGBoost": xgb_savings,
    "ExtraTrees": et_savings,
    "LogisticRegression" : lr_savings
}

# Мета-модель
meta_model = MLPClassifier(hidden_layer_sizes=(20, 10), activation='tanh', max_iter=1000)

# Перебираем все комбинации моделей от 2 до 6
min_comb_size = 2
max_comb_size = len(all_models_m)

results = {}


In [24]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [25]:
results = {}
def combs(min_comb_size, max_comb_size, all_models, X_train, y_train, X_test, y_test):
    target_results=[]
    for r in range(min_comb_size, max_comb_size + 1):
        for model_names in combinations(all_models.keys(), r):
            models = {name: all_models[name] for name in model_names}

            stacking = StackingClassifier(
                estimators=list(models.items()),
                final_estimator=meta_model,
                cv=5,
                stack_method='predict_proba'
            )

            stacking.fit(X_train, y_train)
            y_proba = stacking.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_proba)

            target_results.append({
                "Models": "+".join(model_names),
                "ROC-AUC": auc,
                "Num_Models": r
            })
            print(target_results[-1])
    results[target_name] = pd.DataFrame(target_results).sort_values("ROC-AUC", ascending=False)
print("Mortgage")
combs(min_comb_size, max_comb_size, all_models_m, X_train_m, y_train_m, X_test_m, y_test_m)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Mortgage
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.9424153883613343), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.9421268114961808), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.9410356302248194), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.9406208009811613), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.9422440458476494), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.9430015601186773), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.9422801179557936), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.942153865577289), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.94155867579291), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.9395656918179441), 'Num_Models': 2}
{'Models': 'CatBoost+LightGBM+

In [26]:
print("Pension")
target_name = "Pension"
combs(min_comb_size, max_comb_size, all_models_p,X_train_p, y_train_p, X_test_p, y_test_p)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Pension
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.7778423019648997), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.7782529523272261), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.7783193200625516), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.7762951041351247), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.7780040733197556), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.775789050153268), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.7723337799328857), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.7784105756986242), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.7774648354702363), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.7766435347455835), 'Num_Models': 2}
{'Models': 'CatBoost+LightGBM

In [27]:
print("Savings")
target_name = "Savings"
combs(min_comb_size, max_comb_size, all_models_s,X_train_s, y_train_s, X_test_s, y_test_s)

for target_name, df in results.items():
    print(f"\n🔹 Лучшие комбинации для {target_name}:")
    print(df.head(10).to_markdown(tablefmt="grid"))

Savings
{'Models': 'CatBoost+LightGBM', 'ROC-AUC': np.float64(0.6841728230804244), 'Num_Models': 2}
{'Models': 'CatBoost+XGBoost', 'ROC-AUC': np.float64(0.6915493155001576), 'Num_Models': 2}
{'Models': 'CatBoost+ExtraTrees', 'ROC-AUC': np.float64(0.6895381814362243), 'Num_Models': 2}
{'Models': 'CatBoost+LogisticRegression', 'ROC-AUC': np.float64(0.6956430096985399), 'Num_Models': 2}
{'Models': 'LightGBM+XGBoost', 'ROC-AUC': np.float64(0.6917453870662791), 'Num_Models': 2}
{'Models': 'LightGBM+ExtraTrees', 'ROC-AUC': np.float64(0.6888505304436119), 'Num_Models': 2}
{'Models': 'LightGBM+LogisticRegression', 'ROC-AUC': np.float64(0.6968194390952698), 'Num_Models': 2}
{'Models': 'XGBoost+ExtraTrees', 'ROC-AUC': np.float64(0.6927789643219775), 'Num_Models': 2}
{'Models': 'XGBoost+LogisticRegression', 'ROC-AUC': np.float64(0.6962522320647037), 'Num_Models': 2}
{'Models': 'ExtraTrees+LogisticRegression', 'ROC-AUC': np.float64(0.696140191169777), 'Num_Models': 2}
{'Models': 'CatBoost+LightGBM

In [ ]:
# Базовые модели
base_models_1 = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
}

# Мета-модель
meta_model = LogisticRegression()

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, models, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7598, ROC-AUC = 0.9395
Pension: Accuracy = 0.5540, ROC-AUC = 0.7569
Savings: Accuracy = 0.5521, ROC-AUC = 0.6725


In [ ]:
# Базовые модели
base_models_1 = {
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42),
}

# Мета-модель
meta_model = LogisticRegression()

results = {}
for target_name, (X_train, X_test, y_train, y_test) in targets.items():
    acc, auc = evaluate_stacking(X_train, X_test, y_train, y_test, base_models_1, meta_model)
    results[target_name] = {"Accuracy": acc, "ROC-AUC": auc}

# Вывод результатов
for target, metrics in results.items():
    print(f"{target}: Accuracy = {metrics['Accuracy']:.4f}, ROC-AUC = {metrics['ROC-AUC']:.4f}")

Mortgage: Accuracy = 0.7553, ROC-AUC = 0.9405
Pension: Accuracy = 0.5479, ROC-AUC = 0.7594
Savings: Accuracy = 0.5562, ROC-AUC = 0.6727
